# Lab02 Part 1: Working LAS Files

By Rob Hendrickson

In [1]:
# Import Libraries

# System

import os # For working with Operating System
import requests # For accessing websites
import zipfile # For extracting from Zipfiles
from io import BytesIO # For reading bytes objects

# Analysis

# import laspy # Working with LAS files
import numpy as np # Numerical Python

# # Arcpy Stuff

# import arcpy # Arcpy

# # Set Working Directory (Arcpy)

# arcpy.env.workspace = os.getcwd() + 'Arc1_Lab02.gdb'

## Download Data

Attributes:

    - Dory doesn't like:
        - fields in the spring
        - water unless there's a bridge
            - or it's shallow
    - Optimize gradual slope (gradient is low)

Need: Lidar... This includes classifications:

- 2 Ground
- 5  Vegetation
- 6 – Building
- 8 – Model Keypoint (points required to maintain surface integrity).
- 9 – Water
- 10 – Breakline Proximity. Points within 3 meters of a breakline
- 12 – Overlap Points
- 14 – Bridge Decks

I wonder if we can deduce fields by Vegetation??

Start: 44.127985, -92.148796
End: North Picnic Area (about 44.054364, -92.044832)

In Winona County...

Lidar tiles needed from the `laz` directory found [here](https://resources.gisdata.mn.gov/pub/data/elevation/lidar/county/winona/).
Figured out this list by looking at `tile_index_map.pdf` 

* `['5142-01-63','5142-01-64', '5158-01-01', '5158-01-02', `
* `'4342-32-63','4342-32-64', '4358-32-01', '4358-32-02',  `
* `'4342-31-63','4342-31-64', '4358-31-01', '4358-31-02',  `
* `'4342-30-63','4342-30-64', '4358-30-01']`

In [2]:
# Download Data

tile_indices = ['5142-01-63','5142-01-64', '5158-01-01', '5158-01-02', 
                '4342-32-63','4342-32-64', '4358-32-01', '4358-32-02',  
                '4342-31-63','4342-31-64', '4358-31-01', '4358-31-02',  
                '4342-30-63','4342-30-64', '4358-30-01']

downloaded = True

if not(downloaded):
    # Takes like 5 minutes? Maybe?
    # Total size is 306.8mb

    base_url = 'https://resources.gisdata.mn.gov/pub/data/elevation/lidar/county/winona/'

    for tile_index in tile_indices:

        tile_name = tile_index + '.laz'
        tile_url = base_url + 'laz/' + tile_name

        savepath = os.path.join('Part_2_Tiles', tile_name)

        if not(os.path.exists(savepath)):

            response = requests.request("GET", tile_url) # Get request

            # Save

            with open(savepath, "wb") as file:
                file.write(response.content)

In [3]:
# Laspy doesn't work with laszip files...

# # Make into workable format using laspy
# # see: https://medium.com/spatial-data-science/an-easy-way-to-work-and-visualize-lidar-data-in-python-eed0e028996c

# for tile_index in tile_indices:
    
#     tile_name = tile_index + '.laz'
#     path = os.path.join('Part_2_Tiles', tile_name)

#     las = laspy.read(path)

#     # Make into numpy 3d point cloud tensor

#     point_data = np.stack([las.X, las.Y, las.Z], axis=0).transpose((1, 0))

#     # Unique classifications?
#     classifications = np.unique(np.array(las.classification))
    
#     print(classifications)
    
#     break

In [4]:
# Load into arcpy...

for tile_index in tile_indices:
    
    tile_name = tile_index + '.laz'
    path = os.path.join('Part_2_Tiles', tile_name)

    # las = laspy.read(path)